In [1]:
import json
from bs4 import BeautifulSoup 
import requests 
import re 

In [2]:
filename = "../linux-commits-2023-11-12_random-filtered-1.json"
annotated_commits_list = []
annotated_commits_dict = dict()
with open(filename) as fd:
    for commit in json.load(fd):
        annotated_commits_list.append(commit)
        c_hash = commit['data']['commit']
        annotated_commits_dict[c_hash] = commit

In [3]:
# First commit
print(annotated_commits_list[0]['data']['CommitDate'])

Sat Jan 8 17:44:05 2022 +0100


In [4]:
# Last commit
print(annotated_commits_list[-1]['data']['CommitDate'])

Tue Dec 27 12:07:32 2022 -0600


In [5]:
def getBsoupdocument(url): 
    return BeautifulSoup(getHTMLdocument(url) , 'html.parser')

In [6]:
def getHTMLdocument(url): 
    response = requests.get(url) 
    return response.text

In [7]:
current_releases = (
    ("6.6","v6.x/"),
    ("6.1","v6.x/"),
    ("5.15","v5.x/"),
    ("5.10","v5.x/"),
    ("5.4","v5.x/"),
    ("4.19","v4.x/")
)

In [8]:
base_url = "https://cdn.kernel.org/pub/linux/kernel/"
for exact_release, range_release in current_releases:
    # Change log page (e.g. https://cdn.kernel.org/pub/linux/kernel/v6.x/)
    changelog_list_page = getBsoupdocument(base_url+range_release)
    for link in changelog_list_page.find_all('a', string=re.compile("^ChangeLog-"+exact_release)): 
        # link is "ChangeLog-X.X"
        print(base_url+range_release+link.get('href'))
        # changelog_page_html represent the page of changelog (e.g. https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6)
        changelog_page_html = getHTMLdocument(base_url+range_release+link.get('href'))
        # Find commit hash using regex. Also save if its a normal commit (Author) or a Merge commit (Merge)
        commits_in_changelog = re.findall("commit ([0-9a-f]{5,40})\n(Merge|Author)", changelog_page_html)
        # List to store commits in the changelog that exists in our list of annotated commits
        annotated_commits_found = []
        # Search for changelog commits in our annotated commit collection
        for commit in commits_in_changelog:
            commit_hash = commit[0]
            if commit_hash in annotated_commits_dict:
                annotated_commits_found.append(commit_hash)
        # If at least one commit is found, it is saved in a file "ChangeLog-X.X.X".
        if len(annotated_commits_found) > 0:
            with open('results/%s.txt'%link.get('href'), 'w') as f:
                for commit in annotated_commits_found:
                    f.write("%s\n" % commit)
    #     break # Only first Changelog from now
    # break # Only first from now

https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.1
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.10
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.11
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.12
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.13
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.14
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.15
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.16
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.17
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.18
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.19
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.2
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.20
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.21
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.6.22
https://cdn.k